# $\texttt{GiRaFFEfood}$: Initial data for $\texttt{GiRaFFE}$

## This module provides another initial data option for $\texttt{GiRaFFE}$, drawn from [this paper](https://arxiv.org/abs/1310.3274) .

### NRPy+ Source Code for this module: [GiRaFFEfood_HO/GiRaFFEfood_HO_1D_tests.py](../edit/GiRaFFEfood_HO/GiRaFFEfood_HO_1D_tests.py)

## Introduction:

### Alfv&eacute;n Wave:

 This is a flat-spacetime test with initial data 
\begin{align}
A_x &= 0 \\
A_y &= \left \{ \begin{array}{lll}\gamma_\mu x - 0.015 & \mbox{if} & x \leq -0.1/\gamma_\mu \\
1.15 \gamma_\mu x - 0.03g(x) & \mbox{if} & -0.1/\gamma_\mu \leq x \leq 0.1/\gamma_\mu \\ 
1.3 \gamma_\mu x - 0.015 & \mbox{if} & x \geq 0.1/\gamma_\mu \end{array} \right. , \\
 A_z = &\ y - \gamma_\mu (1-\mu)x ,
\end{align}
which generates the magnetic field in the wave frame,
\begin{align}
B'^{x'}(x') = &\ 1.0,\ B'^y(x') = 1.0, \\
B'^z(x') = &\ \left \{ \begin{array}{lll} 1.0 & \mbox{if} & x' \leq -0.1 \\
				1.0+0.15 f(x') & \mbox{if} & -0.1 \leq x' \leq 0.1 \\
				1.3 & \mbox{if} & x' \geq 0.1 \end{array} \right. ,
\end{align}
and the electric field in the wave frame, 
$$E'^{x'}(x') = -B'^z(0,x') \ \ , \ \ E'^y(x') = 0.0 \ \ , \ \ E'^z(x') = 1.0  .$$

These are converted to the grid frame by 
\begin{align}
  B^x(0,x) = &\ B'^{x'}(\gamma_\mu x) , \\
  B^y(0,x) = &\ \gamma_\mu [ B'^y(\gamma_\mu x) - \mu E'^z(\gamma_\mu x) ] , \\ 
  B^z(0,x) = &\ \gamma_\mu [ B'^z(\gamma_\mu x) + \mu E'^y(\gamma_\mu x) ] , 
\end{align}
and
\begin{align}
  E^x(0,x) = &\ E'^{x'}(\gamma_\mu x) , \\ 
  E^y(0,x) = &\ \gamma_\mu [ E'^y(\gamma_\mu x) + \mu B'^z(\gamma_\mu x) ] ,\\ 
  E^z(0,x) = &\ \gamma_\mu [ E'^z(\gamma_\mu x) - \mu B'^y(\gamma_\mu x) ],
\end{align}
and the velocity is given by $$\mathbf{v} = \frac{\mathbf{E} \times \mathbf{B}}{B^2}$$ in flat spacetime. Additionally, $f(x)=1+\sin (5\pi x)$, $-1<\mu<1$ is the wave speed relative to the grid frame and $\gamma_\mu = (1-\mu^2)^{-1/2}$, and $g(x) = \cos (5\pi \gamma_\mu x)/\pi$.

For the eventual purpose of testing convergence, any quantity $Q$ evolves as $Q(t,x) = Q(0,x-\mu t)$

See the [Tutorial-GiRaFFEfood_HO](Tutorial-GiRaFFEfood_HO.ipynb) tutorial module for more general detail on how this is used.


<a id='toc'></a>

# Table of Contents:
$$\label{toc}$$

This module is organized as follows

1. [Step 1](#initializenrpy): Import core NRPy+ modules and set NRPy+ parameters 
1. [Step 2](#vector_ak): Set the vector $A_k$
1. [Step 3](#vectors_for_velocity): Set the vectors $B^i$ and $E^i$ for the velocity
1. [Step 4](#vi): Calculate $v^i$
1. [Step 5](#code_validation): Code Validation against GiRaFFEfood_HO/GiRaFFEfood_HO_1D_tests NRPy+ module
1. [Step 6](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Import core NRPy+ modules and set NRPy+ parameters \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Here, we will import the NRPy+ core modules and set the reference metric to Cartesian, set commonly used NRPy+ parameters, and set C parameters that will be set from outside the code eventually generated from these expressions. We will also set up a parameter to determine what initial data is set up, although it won't do much yet.

In [1]:
# Step 0: Import the NRPy+ core modules and set the reference metric to Cartesian
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import loop

import reference_metric as rfm
par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()

# Step 1a: Set commonly used parameters.
thismodule = "GiRaFFEfood_HO_1D"
# Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")


<a id='vector_ak'></a>

# Step 2: Set the vector $A_k$ \[Back to [top](#toc)\]
$$\label{vector_ak}$$

The vector potential is given as
\begin{align}
A_x &= 0 \\
A_y &= \left \{ \begin{array}{lll}\gamma_\mu x - 0.015 & \mbox{if} & x \leq -0.1/\gamma_\mu \\
1.15 \gamma_\mu x - 0.03g(x) & \mbox{if} & -0.1/\gamma_\mu \leq x \leq 0.1/\gamma_\mu \\ 
1.3 \gamma_\mu x - 0.015 & \mbox{if} & x \geq 0.1/\gamma_\mu \end{array} \right. , \\
A_z &= y - \gamma_\mu (1-\mu)x .
\end{align}
First, however, we must set $$\gamma_\mu = (1-\mu^2)^{-1/2}$$ and $$g(x) = \cos (5\pi \gamma_\mu x)/\pi$$.


In [2]:
mu_AW,M_PI = par.Cparameters("REAL",thismodule,["mu_AW","M_PI"]) # The wave speed and pi in C
gammamu = 1/sp.sqrt(1-mu_AW**2)

# We'll use reference_metric.py to define x and y
x = rfm.xxCart[0]
y = rfm.xxCart[1]

g_AW = sp.cos(5*M_PI*gammamu*x)/M_PI

Now, we can define the vector potential. We will create three copies of this variable, because the potential is uniquely defined in three zones. Data for $x \leq -0.1/\gamma_\mu$ shall be referred to as "left", data for $-0.1/\gamma_\mu \leq x \leq 0.1/\gamma_\mu$ as "center", and data for $x \geq 0.1/\gamma_\mu$ as "right".

Starting on the left, 
\begin{align}
A_x &= 0 \\
A_y &= \gamma_\mu x - 0.015 \\
A_z &= y - \gamma_\mu (1-\mu)x .
\end{align}

In [3]:
AD = ixp.register_gridfunctions_for_single_rank1("EVOL","AD")
AleftD = ixp.zerorank1()

AleftD[0] = sp.sympify(0)
AleftD[1] = gammamu*x-0.015
AleftD[2] = y-gammamu*(1-mu_AW)*x

In the the center,
\begin{align}
A_x &= 0 \\
A_y &= 1.15 \gamma_\mu x - 0.03g(x) \\
A_z &= y - \gamma_\mu (1-\mu)x .
\end{align}

In [4]:
AcenterD = ixp.zerorank1()

AcenterD[0] = sp.sympify(0)
AcenterD[1] = 1.15*gammamu*x-0.03*g_AW
AcenterD[2] = y-gammamu*(1-mu_AW)*x

And on the right,
\begin{align}
A_x &= 0 \\
A_y &= 1.3 \gamma_\mu x - 0.015 \\
A_z &= y - \gamma_\mu (1-\mu)x .
\end{align}

In [5]:
ArightD = ixp.zerorank1()

ArightD[0] = sp.sympify(0)
ArightD[1] = 1.3*gammamu*x-0.015
ArightD[2] = y-gammamu*(1-mu_AW)*x

<a id='vectors_for_velocity'></a>

# Step 3: Set the vectors $B^i$ and $E^i$ for the velocity \[Back to [top](#toc)\]
$$\label{vectors_for_velocity}$$

Now, we will set the magnetic and electric fields that we will need to define the initial velocities. First, we need to define $$f(x)=1+\sin (5\pi x);$$ note that in the definition of $B^i$, we need $f(x')$ where $x'=\gamma_\mu x$.


In [6]:
xprime = gammamu*x
f_AW = 1.0 + sp.sin(5.0*M_PI*xprime)
print(f_AW)

sin(5.0*M_PI*xx0/sqrt(-mu_AW**2 + 1)) + 1.0


We will now set the magnetic field in the wave frame:
\begin{align}
B'^{x'}(x') = &\ 1.0,\ B'^y(x') = 1.0, \\
B'^z(x') = &\ \left \{ \begin{array}{lll} 1.0 & \mbox{if} & x' \leq -0.1 \\
				1.0+0.15 f(x') & \mbox{if} & -0.1 \leq x' \leq 0.1 \\
				1.3 & \mbox{if} & x' \geq 0.1 \end{array} \right. .
\end{align}


In [7]:
BleftpU = ixp.zerorank1()
BleftpU[0] = sp.sympify(1.0)
BleftpU[1] = sp.sympify(1.0)
BleftpU[2] = sp.sympify(1.0)

BcenterpU = ixp.zerorank1()
BcenterpU[0] = sp.sympify(1.0)
BcenterpU[1] = sp.sympify(1.0)
BcenterpU[2] = 1.0 + 0.15*f_AW

BrightpU = ixp.zerorank1()
BrightpU[0] = sp.sympify(1.0)
BrightpU[1] = sp.sympify(1.0)
BrightpU[2] = sp.sympify(1.3)


Now, we will set the electric field in the wave frame:
\begin{align}
E'^{x'}(x') &= -B'^z(0,x'), \\ 
E'^y(x') &= 0.0, \\ 
E'^z(x') &= 1.0  .
\end{align}

In [8]:
EleftpU = ixp.zerorank1()
EleftpU[0] = -BleftpU[2]
EleftpU[1] = sp.sympify(0.0)
EleftpU[2] = sp.sympify(1.0)

EcenterpU = ixp.zerorank1()
EcenterpU[0] = -BcenterpU[2]
EcenterpU[1] = sp.sympify(0.0)
EcenterpU[2] = sp.sympify(1.0)

ErightpU = ixp.zerorank1()
ErightpU[0] = -BrightpU[2]
ErightpU[1] = sp.sympify(0.0)
ErightpU[2] = sp.sympify(1.0)


Next, we must transform the the fields into the grid frame. We'll do the magnetic fields first.
\begin{align}
  B^x(0,x) = &\ B'^{x'}(\gamma_\mu x) , \\
  B^y(0,x) = &\ \gamma_\mu [ B'^y(\gamma_\mu x) - \mu E'^z(\gamma_\mu x) ] , \\ 
  B^z(0,x) = &\ \gamma_\mu [ B'^z(\gamma_\mu x) + \mu E'^y(\gamma_\mu x) ] , 
\end{align}


In [9]:
BleftU = ixp.zerorank1()
BleftU[0] = BleftpU[0]
BleftU[1] = gammamu*(BleftpU[1]-mu_AW*EleftpU[2])
BleftU[2] = gammamu*(BleftpU[2]+mu_AW*EleftpU[1])

BcenterU = ixp.zerorank1()
BcenterU[0] = BcenterpU[0]
BcenterU[1] = gammamu*(BcenterpU[1]-mu_AW*EcenterpU[2])
BcenterU[2] = gammamu*(BcenterpU[2]+mu_AW*EcenterpU[1])

BrightU = ixp.zerorank1()
BrightU[0] = BrightpU[0]
BrightU[1] = gammamu*(BrightpU[1]-mu_AW*ErightpU[2])
BrightU[2] = gammamu*(BrightpU[2]+mu_AW*ErightpU[1])
print(BrightU[1])

(-1.0*mu_AW + 1.0)/sqrt(-mu_AW**2 + 1)


And now the electric fields:
\begin{align}
  E^x(0,x) = &\ E'^{x'}(\gamma_\mu x) , \\ 
  E^y(0,x) = &\ \gamma_\mu [ E'^y(\gamma_\mu x) + \mu B'^z(\gamma_\mu x) ] ,\\ 
  E^z(0,x) = &\ \gamma_\mu [ E'^z(\gamma_\mu x) - \mu B'^y(\gamma_\mu x) ],
\end{align}


In [10]:
EleftU = ixp.zerorank1()
EleftU[0] = EleftpU[0]
EleftU[1] = gammamu*(EleftpU[1]+mu_AW*BleftpU[2])
EleftU[2] = gammamu*(EleftpU[2]-mu_AW*BleftpU[1])

EcenterU = ixp.zerorank1()
EcenterU[0] = EcenterpU[0]
EcenterU[1] = gammamu*(EcenterpU[1]+mu_AW*BcenterpU[2])
EcenterU[2] = gammamu*(EcenterpU[2]-mu_AW*BcenterpU[1])

ErightU = ixp.zerorank1()
ErightU[0] = ErightpU[0]
ErightU[1] = gammamu*(ErightpU[1]+mu_AW*BrightpU[2])
ErightU[2] = gammamu*(ErightpU[2]-mu_AW*BrightpU[1])


<a id='vi'></a>

# Step 4: Calculate $v^i$ \[Back to [top](#toc)\]
$$\label{vi}$$

Now, we calculate $$\mathbf{v} = \frac{\mathbf{E} \times \mathbf{B}}{B^2},$$ which is equivalent to $$v^i = [ijk] \frac{E^j B^k}{B^2},$$ where $[ijk]$ is the Levi-Civita symbol and $B^2 = \gamma_{ij} B^i B^j$ is a trivial dot product in flat space.


In [11]:
import WeylScal4NRPy.WeylScalars_Cartesian as weyl
LeviCivitaSymbolDDD = weyl.define_LeviCivitaSymbol_rank3()

Bleft2 = BleftU[0]*BleftU[0] + BleftU[1]*BleftU[1] + BleftU[2]*BleftU[2]
Bcenter2 = BcenterU[0]*BcenterU[0] + BcenterU[1]*BcenterU[1] + BcenterU[2]*BcenterU[2]
Bright2 = BrightU[0]*BrightU[0] + BrightU[1]*BrightU[1] + BrightU[2]*BrightU[2]

ValenciavU = ixp.register_gridfunctions_for_single_rank1("AUX","ValenciavU")

ValenciavleftU = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            ValenciavleftU[i] += LeviCivitaSymbolDDD[i][j][k] * EleftU[j] * BleftU[k] / Bleft2
            
ValenciavcenterU = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            ValenciavcenterU[i] += LeviCivitaSymbolDDD[i][j][k] * EcenterU[j] * BcenterU[k] / Bcenter2
            
ValenciavrightU = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            ValenciavrightU[i] += LeviCivitaSymbolDDD[i][j][k] * ErightU[j] * BrightU[k] / Bright2

<a id='code_validation'></a>

# Step 5: Code Validation against GiRaFFEfood_HO/GiRaFFEfood_HO_1D_tests NRPy+ module \[Back to [top](#toc)\]
$$\label{code_validation}$$

Here, as a code validation check, we verify agreement in the SymPy expressions for the $\texttt{GiRaFFE}$ Aligned Rotator initial data equations  we intend to use between
1. this tutorial and 
2. the NRPy+ [GiRaFFEfood_HO_1D_tests.py](../edit/GiRaFFEfood_HO/GiRaFFEfood_HO_1D_tests.py) module.



In [12]:
# Reset the list of gridfunctions, as registering a gridfunction
#   twice will spawn an error.
gri.glb_gridfcs_list = []

print("vvv Ignore the minor warnings below. vvv")

import GiRaFFEfood_HO.GiRaFFEfood_HO_1D_tests as gfho
gfho.GiRaFFEfood_HO_1D_tests()

print("^^^ Ignore the minor warnings above. ^^^\n")

print("Consistency check between GiRaFFEfood_HO tutorial and NRPy+ module: ALL SHOULD BE ZERO.")

for i in range(DIM):

    print("ValenciavleftU["+str(i)+"] - gfho.ValenciavleftU["+str(i)+"] = " + str(ValenciavleftU[i] - gfho.ValenciavleftU[i]))
    print("AleftD["+str(i)+"] - gfho.AleftD["+str(i)+"] = " + str(AleftD[i] - gfho.AleftD[i]))
    print("ValenciavcenterU["+str(i)+"] - gfho.ValenciavcenterU["+str(i)+"] = " + str(ValenciavcenterU[i] - gfho.ValenciavcenterU[i]))
    print("AcenterD["+str(i)+"] - gfho.AcenterD["+str(i)+"] = " + str(AcenterD[i] - gfho.AcenterD[i]))
    print("ValenciavrightU["+str(i)+"] - gfho.ValenciavrightU["+str(i)+"] = " + str(ValenciavrightU[i] - gfho.ValenciavrightU[i]))
    print("ArightD["+str(i)+"] - gfho.ArightD["+str(i)+"] = " + str(ArightD[i] - gfho.ArightD[i]))


vvv Ignore the minor warnings below. vvv
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::M_PI
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::xmin
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::xmax
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::ymin
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::ymax
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::zmin
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::zmax
initialize_param() minor warning: Did nothing; already initialized parameter GiRaFFEfood_HO_1D::mu_AW
initialize_param() minor warning: Did nothing; already initialized parameter GiRaFFEfood_HO_1D::M_PI
^^^ Ignore the minor warnings above. ^^^

Consistency ch

<a id='latex_pdf_output'></a>

# Step 6: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFEfood_HO_1D_tests.pdf](Tutorial-GiRaFFEfood_HO_1D_tests.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [13]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-GiRaFFEfood_HO_1D_tests.ipynb
!pdflatex -interaction=batchmode Tutorial-GiRaFFEfood_HO_1D_tests.tex
!pdflatex -interaction=batchmode Tutorial-GiRaFFEfood_HO_1D_tests.tex
!pdflatex -interaction=batchmode Tutorial-GiRaFFEfood_HO_1D_tests.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-GiRaFFEfood_HO_1D_tests.ipynb to latex
[NbConvertApp] Writing 51215 bytes to Tutorial-GiRaFFEfood_HO_1D_tests.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
